In [1]:
# Cell 1: Installation
!pip install owlready2


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from owlready2 import *
import json
import re


# We define namespaces
dul_onto = get_ontology("http://www.ease-crc.org/ont/DUL.owl").load()
dul_ns = get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
onto = get_ontology("http://www.semanticweb.org/gamematcher.owl")
onto.imported_ontologies.append(dul_onto)

with onto:
    onto.metadata.comment.append("Andreea Scrob, Edoardo Tommasi")
    onto.metadata.comment.append("GameMatcher is an intelligent companion system that assists users in finding video games compatible with their computer.")
    onto.metadata.label.append("GameMatcher Ontology")

    # TAXONOMY (CLASSES)
    #  USER
    class User(Thing):
        comment = ["An Agent who interacts with the system to find compatible video games."]
        label = ["User"]
        is_a = [dul_ns.Agent]

    # COMPUTER and COMPONENTS
    # Note: A Computer is a Physical Object, not a subclass of Component.
    class Computer(Thing):
        comment = ["A Physical Object consisting of hardware components capable of executing software."]
        label = ["Computer"]
        is_a = [dul_ns.PhysicalObject]

    class Component(Thing):
        comment = ["A Physical Object that forms a constituent part of a computing system."]
        label = ["Component"]
        is_a = [dul_ns.PhysicalObject]

    # Specific Hardware Classes
    class CPU(Component):
        comment = ["A Hardware Component that acts as the primary processing unit."]
        label = ["CPU"]

    class GPU(Component):
        comment = ["A Hardware Component specialized for rendering graphics."]
        label = ["GPU"]

    class RAM(Component):
        comment = ["A Hardware Component used for temporary data storage."]
        label = ["RAM"]

    class Storage(Component):#da specificare nella relazione che questo diventa Storage poi lo spazio disponibile viene preso con la dataproperty
        comment = ["A Hardware Component used for data retention (e.g., HDD or SSD)."]
        label = ["Storage"]

    # Specific Software Classes
    class OperatingSystem(Thing):
        comment = ["A Software Component that manages computer hardware and resources."]
        label = ["Operating System"]
        is_a = [dul_ns.InformationObject]

    #  VIDEOGAME and REQUIREMENTS
    class VideoGame(Thing):
        comment = ["An Information Object representing a digital game available for play."]
        label = ["Video Game"]
        is_a = [dul_ns.InformationObject]

    class MinimumRequirement(Thing):
        comment = ["A Requirement specifying the absolute minimum specifications for a game to run."]
        label = ["Minimum Requirement"]
        is_a = [dul_ns.Description]


    #  ATTRIBUTES and CONCEPTS
    #todo: chiedere al prof se possiamo aggiungere GameAttribute
    class GameAttribute(Thing):
        comment = ["A Concept representing a characteristic feature of a video game."]
        label = ["Game Attribute"]
        is_a = [dul_ns.Concept]

    class GameGenre(GameAttribute):
        comment = ["A classification based on gameplay interaction (e.g., RPG, Strategy)."]
        label = ["Game Genre"]

    class PlayerMode(GameAttribute):
        comment = ["Defines the number of players supported (e.g., Single-player)."]
        label = ["Player Mode"]

    class PEGIRating(GameAttribute):
        comment = ["Indicates the suitability of content for different age groups."]
        label = ["PEGI Rating"]

    # 1. Top-Level Disjointness
    # A User is not a Computer, which is not a Game, etc.
    AllDisjoint([User, Computer, Component, OperatingSystem, VideoGame, MinimumRequirement, GameAttribute])

    # 2. Hardware Component Disjointness
    # A CPU is not a GPU, RAM, or Storage.
    AllDisjoint([CPU, GPU, RAM, Storage])

    # 3. Attribute Disjointness
    # A Genre is not a PEGI Rating or a Player Mode.
    AllDisjoint([GameGenre, PlayerMode, PEGIRating])


    # OBJECT PROPERTIES (RELATIONS)
    # Relations: Computer -> Components
    class hasComponent(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]
        range = [Component]
        comment = ["Generic relation linking a computer to its parts."]
        label = ["has component"]


    # Sub-properties for specificity
    class hasCPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [CPU]
        comment = ["Relation linking components to the computer's cpu."]
        label = ["has CPU"]

    class hasGPU(hasComponent, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [GPU]
        comment = ["Relation linking components to the computer's gpu."]
        label = ["has GPU"]

    class hasRAM(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [RAM]
        comment = ["Relation linking components to the computer's ram."]
        label = ["has RAM"]

    class hasStorage(hasComponent, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [Storage]
        comment = ["Relation linking components to the computer's  available storage."]
        label = ["has storage"]

    class hasOS(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [Computer]; range = [OperatingSystem]
        comment = ["Relation linking components to the computer's os."]
        label = ["has operating system"]


    # Relations: Videogame -> Requirements
    class hasMinRequirement(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [MinimumRequirement]
        comment = ["Relation linking videogame to its component requirements to make it run."]
        label = ["has minimum requirement"]


    # Relations: User -> Computer
    class hasComputer(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]
        range = [Computer]
        comment = ["Relation linking user to the computer he owns."]
        label = ["has computer device"]



    # Relations: Attribute
    class hasGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [GameGenre]
        comment = ["Relation linking videogame to its genre."]
        label = ["has genre"]


    class preferredGenre(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [GameGenre]
        comment = ["Relation linking user to its preferred game's genre."]
        label = ["prefers genre"]

    class hasPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PlayerMode]
        comment = ["Relation linking videogame to its player mode."]
        label = ["has player mode"]


    class preferredPlayerMode(ObjectProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [User]; range = [PlayerMode]
        comment = ["Relation linking user to its preferred game mode."]
        label = ["prefers player mode"]


    class hasPEGI(ObjectProperty, FunctionalProperty, AsymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]; range = [PEGIRating]
        comment = ["Relation linking videogame to its age rating."]
        label = ["has PEGI rating"]


    # Derived Relation (for Compatibility Checking)
    class isCompatibleWith(ObjectProperty, SymmetricProperty, IrreflexiveProperty):
        domain = [VideoGame]
        range = [Computer]
        comment = ["Inferred relation: true if the computer meets all game requirements."]
        label = ["is compatible with"]



    # DATA PROPERTIES (ACTIONABLE LOGIC)

    # BENCHMARK SCORE (CPU and GPU)


    # Game Requirement Property
    class requiresCPUBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        label = ["requires CPU benchmark score"]

    class requiresGPUBenchmarkScore(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [int]
        label = ["requires GPU benchmark score"]


    #  MEMORY (RAM and VRAM)
    class hasMemorySizeGB(DataProperty, FunctionalProperty):
        domain = [RAM, GPU]
        range = [float]
        comment = ["The amount of memory in Gigabytes."]
        label = ["has memory size (GB)"]


    class requiresMemoryGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum RAM or VRAM required in Gigabytes."]
        label = ["requires memory (GB)"]


    #  STORAGE
    class hasStorageSizeGB(DataProperty, FunctionalProperty):
        domain = [Storage]
        range = [float]
        comment = ["Total available disk space in Gigabytes."]
        label = ["has storage size (GB)"]


    class requiresStorageSpaceGB(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Installation size required in Gigabytes."]
        label = ["requires storage space (GB)"]


    # OS LOGIC
    class hasOSVersionValue(DataProperty, FunctionalProperty):
        domain = [OperatingSystem]
        range = [float]
        comment = ["Numeric version of the OS (e.g., 10.0, 11.0)."]
        label = ["has OS version value"]


    class requiresMinOSVersionValue(DataProperty, FunctionalProperty):
        domain = [MinimumRequirement]
        range = [float]
        comment = ["Minimum numeric OS version required."]
        label = ["requires min OS version"]



   #  BUDGET and PRICING
    class hasPrice(DataProperty, FunctionalProperty):
        domain = [VideoGame]
        range = [float]
        comment = ["The monetary cost required to purchase the digital game."]
        label = ["has price"]


    class hasBudget(DataProperty, FunctionalProperty):
        domain = [User]
        range = [float]
        comment = ["The maximum monetary limit the user is willing to spend."]
        label = ["has budget"]


    #  AGE and PEGI
    class hasAge(DataProperty, FunctionalProperty):
        domain = [User]
        range = [int]
        comment = ["The chronological age of the user (in years), used for PEGI checks."]
        label = ["has age"]


    class hasPEGIAgeThreshold(DataProperty, FunctionalProperty):
        domain = [PEGIRating]
        range = [int]
        comment = ["The numeric age threshold (e.g., 18)."]
        label = ["has PEGI threshold"]




In [2]:

# FUNZIONI DI UTILITÀ (Parsing & Matching)

def clean_iri(text):
    """Rimuove caratteri illegali per creare un IRI valido."""
    if not text: return "Unknown"
    # Sostituisce spazi e caratteri strani con underscore
    clean = re.sub(r'[^a-zA-Z0-9]', '_', str(text))
    # Rimuove underscore multipli
    clean = re.sub(r'_+', '_', clean)
    return clean.strip('_')

def parse_gb(text):
    """Estrae i GB da stringhe come '8 GB RAM' o '100 GB available'."""
    if not text: return 0.0
    match = re.search(r'(\d+)', str(text))
    if match:
        return float(match.group(1))
    return 0.0

def parse_price(text):
    """Estrae il prezzo da stringhe come '39,99€' o 'Free To Play'."""
    if not text: return 0.0
    if "Free" in text: return 0.0
    # Sostituisce virgola con punto ed estrae numeri
    text = text.replace(',', '.')
    match = re.search(r'(\d+\.\d+)', text)
    if match:
        return float(match.group(1))
    return 0.0

def parse_os_version(os_string):
    if not os_string or os_string == "N/A":
        return None
    # Look for numbers like 10, 11, or 7 in the string
    match = re.search(r'(\d+)', os_string)
    if match:
        return float(match.group(1))
    return None

def find_hardware_score(req_text, hardware_db):
    """
    Cerca il punteggio benchmark.
    req_text: Stringa da Steam (es. 'NVIDIA GeForce GTX 1060')
    hardware_db: Lista di dizionari dal JSON benchmark
    """
    if not req_text: return None
    
    req_clean = req_text.lower().replace('-', ' ').replace('_', ' ')
    
    best_match = None
    best_score = 0
    
    # Logica semplice: se il nome del benchmark è contenuto nel requisito
    for item in hardware_db:
        bench_name = item['name'].lower().replace('-', ' ')
        # Esempio: "geforce gtx 1060" in "nvidia geforce gtx 1060" -> VERO
        if bench_name in req_clean:
            # Troviamo il match più lungo (solitamente più preciso)
            if best_match is None or len(bench_name) > len(best_match):
                best_match = bench_name
                best_score = int(item['mark'])
    
    return best_score

In [ ]:
import json
import re
from pathlib import Path

# --- UNIVERSAL PATH SETUP ---
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "Scraping" / "cleaning"

try:
    with open(DATA_DIR / "cpu_filtered.json", 'r') as f: cpu_data = json.load(f)
    with open(DATA_DIR / "gpu_filtered.json", 'r') as f: gpu_data = json.load(f)
    with open(DATA_DIR / "steam_parsed_CLEAN.json", 'r') as f: game_data = json.load(f)
    print("File JSON caricati correttamente.")
except FileNotFoundError:
    print(f"Errore: File non trovati in {DATA_DIR}")
    cpu_data, gpu_data, game_data = [], [], []

# --- ONTOLOGY POPULATION ---
with onto:
    print("Popolamento Hardware...")
    cpu_db = cpu_data 
    gpu_db = gpu_data
    
    # 1. Create Hardware Instances
    for cpu in cpu_data:
        cpu_iri = clean_iri(cpu['name'])
        new_cpu = onto.CPU(f"CPU_{cpu_iri}")
        new_cpu.label = [cpu['name']]
        new_cpu.hasBenchmarkScore = int(cpu['mark'])
        
    for gpu in gpu_data:
        gpu_iri = clean_iri(gpu['name'])
        new_gpu = onto.GPU(f"GPU_{gpu_iri}")
        new_gpu.label = [gpu['name']]
        new_gpu.hasBenchmarkScore = int(gpu['mark'])

    print("Popolamento Giochi...")
    for game in game_data:
        # Game Basic Info
        game_name = game.get('NAME', 'UnknownGame')
        game_iri = clean_iri(game_name)
        new_game = onto.VideoGame(f"Game_{game_iri}")
        new_game.label = [game_name]
        
        # Price & PEGI
        new_game.hasPrice = parse_price(game.get('PRICE'))
        pegi_text = game.get('PEGI', 'N/A')
        if pegi_text and pegi_text != 'N/A' and not pegi_text.startswith('#'):
            pegi_iri = clean_iri(pegi_text)
            pegi_instance = onto.PEGIRating(f"PEGI_{pegi_iri}")
            pegi_instance.label = [pegi_text]
            age_match = re.search(r'\d+', pegi_text)
            if age_match:
                pegi_instance.hasPEGIAgeThreshold = int(age_match.group(0))
            new_game.hasPEGI = pegi_instance

        # Player Modes
        mode_text = game.get('MODE', '')
        if mode_text:
            modes = [m.strip() for m in mode_text.split(',')]
            for m in modes:
                mode_iri = clean_iri(m)
                mode_instance = onto.PlayerMode(f"Mode_{mode_iri}")
                mode_instance.label = [m]
                new_game.hasPlayerMode.append(mode_instance)

       # --- CREATE UNIQUE REQUIREMENT INDIVIDUAL ---
        # Named Req_Game_Name (e.g., Req_Cyberpunk_2077)
        req_iri = f"Req_{game_iri}"
        new_req = onto.MinimumRequirement(req_iri)
        
        # Explicitly link it to the game
        new_game.hasMinRequirement = new_req
        
        # Set a clear label for the requirement individual
        new_req.label = [f"Minimum Requirements for {game_name}"]

        # --- NEW: OS LOGIC ---
        os_text = game.get('OS', '')
        os_version = parse_os_version(os_text)
        if os_version:
            new_req.requiresMinOSVersionValue = os_version
            new_req.label.append(f"OS Version Required: {os_version}")

        # --- GPU DATA (Stored INSIDE new_req) ---
        valid_gpus = []
        for k in ['GPU_0', 'GPU_1', 'GPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, gpu_db)
            if score and score > 0:
                valid_gpus.append((int(score), val))
        
        if valid_gpus:
            valid_gpus.sort(key=lambda x: x[0])
            best_score, best_name = valid_gpus[0]
            # We save the score as a DataProperty on the Requirement individual
            new_req.requiresGPUBenchmarkScore = best_score
            

        # --- CPU DATA (Stored INSIDE the SAME new_req) ---
        valid_cpus = []
        for k in ['CPU_0', 'CPU_1', 'CPU_2']:
            val = game.get(k)
            score = find_hardware_score(val, cpu_db)
            if score and score > 0:
                valid_cpus.append((int(score), val))
        
        if valid_cpus:
            valid_cpus.sort(key=lambda x: x[0])
            best_score, best_name = valid_cpus[0]
            # We save the score as a DataProperty on the same Requirement individual
            new_req.requiresCPUBenchmarkScore = best_score
            

print("ABox creata con successo!")
onto.save(file="gameMatcher_ABox.owl")
print(f"Ontologia salvata in: gameMatcher_ABox.owl")

File JSON caricati correttamente.
Popolamento Hardware...
Popolamento Giochi...
ABox creata con successo!
Ontologia salvata in: gameMatcher_ABox.owl
